# Pytorch의 nn.Embedding
- Pytorch의 Embedding Layer는 word2vec과 마찬가지로 word embedding vector를 찾는 **Lookup Table**이다.
    - 단어의 **정수의 고유 index**가 입력으로 들어오면 Embedding Layer의 **그 index의 Vector**를 출력한다.
    - 모델이 학습되는 동안 모델이 풀려는 문제에 맞는 값으로 Embedding Layer의 vector들이 업데이트 된다.
    - Word2Vec의 embedding vector 학습을 nn.Embedding은 자신이 포함된 모델을 학습 하는 과정에서 한다고 생각하면 된다.

In [11]:
import torch
import torch.nn as nn

embedding_model = nn.Embedding(
    num_embeddings=20000, # vocab size(어휘사전의 어휘개수): 몇개 단어(토큰)에 대한 Embedding vector를 만들지 설정.
    embedding_dim=10, # Embedding vector의 차원수
    padding_idx=0,      
    # padding 토큰의 index. padding 토큰은 글자수를 맞추기 위해 채우는 값(0) 그래서 embedding값을 학습할 필요가 없다.
)
# 20000 x 200
# nn.GRU(input_size=200)

In [12]:
# 파라미터 확인
weight = embedding_model.weight
weight.shape
weight[0] # 0번 토큰(단어)의 embeding vector값을 조회
# weight[1]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], grad_fn=<SelectBackward0>)

In [13]:
# "나는-30 어제-100 밥을-600 먹었다-7200 .- 5"
# 문장을 tokenizer를 통해 토큰화 한 결과.
doc_token_ids = torch.tensor([[30, 100, 600, 7200, 5]], dtype=torch.int64)  # 한문장
doc_token_ids = torch.tensor([[30, 100, 600, 7200, 5],[30, 100, 600, 7200, 5],[30, 100, 600, 7200, 5]], dtype=torch.int64)  # 여러문장

doc_embedding_vector = embedding_model(doc_token_ids)

doc_embedding_vector.shape # [1: batch_size-1문장, 5: seq_len-토큰개수, 200: embedding vector]
# [1, 5, 200], [batch_size, seq_length, embedding_vector_size]

torch.Size([3, 5, 10])

In [15]:
doc_embedding_vector[0][1]

tensor([ 0.0763,  0.9269, -1.7806, -0.3793,  0.6069, -1.9952,  0.3360,  0.6979,
        -1.2362, -1.2926], grad_fn=<SelectBackward0>)

# 네이버 영화 댓글 감성분석(Sentiment Analysis)

## 감성분석(Sentiment Analysis) 이란
입력된 텍스트가 **긍적적인 글**인지 **부정적인**인지 또는 **중립적인** 글인지 분석하는 것을 감성(감정) 분석이라고 한다.   
이를 통해 기업이 고객이 자신들의 기업 또는 제품에 대해 어떤 의견을 가지고 있는지 분석한다.

# Dataset, DataLoader 생성

## Korpora에서 Naver 영화 댓글 dataset 가져오기
- https://github.com/ko-nlp/Korpora
- http://github.com/e9t/nsmc/
    - input: 영화댓글
    - output: 0(부정적댓글), 1(긍정적댓글)
### API
- **corpus 가져오기**
    - `Korpora.load('nsmc')`
- **text/label 조회**
    - `corpus.get_all_texts()` : 전체 corpus의 text들을 tuple로 반환
    - `corpus.get_all_labels()`: 전체 corpus의 label들을 list로 반환
- **train/test set 나눠서 조회**
    - `corpus.train`
    - `corpus.test`
    - `LabeledSentenceKorpusData` 객체에 text와 label들을 담아서 제공.
        - `LabeledSentenceKorpusData.texts`: text들 tuple로 반환.
        - `LabeledSentenceKorpusData.labels`: label들 list로 반환.

## 데이터 로딩

In [16]:
from Korpora import Korpora
corpus = Korpora.load('nsmc')


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\Playdata\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\

In [17]:
all_inputs = corpus.get_all_texts() # X : 댓글
all_labels = corpus.get_all_labels() # y: label (0: 부정적, 1: 긍적적)

In [18]:
all_inputs[:5]

('아 더빙.. 진짜 짜증나네요 목소리',
 '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나',
 '너무재밓었다그래서보는것을추천한다',
 '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정',
 '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다')

In [19]:
all_labels[:5]

[0, 1, 0, 0, 1]

In [20]:
len(all_inputs)

200000

In [22]:
print(type(corpus.train))
corpus.train

<class 'Korpora.korpora.LabeledSentenceKorpusData'>


NSMC.train: size=150000
  - NSMC.train.texts : list[str]
  - NSMC.train.labels : list[int]

In [25]:
corpus.train.texts[:5]
corpus.train.labels[:5]

[0, 1, 0, 0, 1]

In [26]:
corpus.test

NSMC.test: size=50000
  - NSMC.test.texts : list[str]
  - NSMC.test.labels : list[int]

## 토큰화
1. 형태소 단위 token화
    - konlpy로 token화 한 뒤 다시 한 문장으로 만든다.
2. 1에서 처리한 corpus를 BPE 로 token화
   
### 전처리 함수

#### 형태소 단위 분절

In [27]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
# from konlpy.tag import Okt
from kiwipiepy import Kiwi
import string
import re

kiwi = Kiwi()
def text_preprocessing(text):
    """
    1. 영문 -> 소문자로 변환
    2. 구두점 제거
    3. 형태소 기반 토큰화
    4. 형태소로 토큰화 한 뒤 다시 하나의 문자열로 묶어서 반환.
    """
    text = text.lower()
    text = re.sub(rf"[{string.punctuation}]", ' ', text) # 구두점(특수문자)들을 ' '으로 변환.
    text = [token.lemma for token in kiwi.tokenize(text)] # [a, b, c,]
    return ' '.join(text)

In [30]:
print(all_inputs[100])
text_preprocessing(all_inputs[100])

신카이 마코토의 작화와,미유와 하나카나가 연기를 잘해줘서 더대박이였다.


'신카이 마코토 의 작화 와 미유 와 하나카나 가 연기 를 잘 하다 어 주다 어서 더 대박 이다 였 다'

In [31]:
train_texts = corpus.train.texts
train_inputs = [text_preprocessing(txt) for txt in train_texts]

test_texts = corpus.test.texts
test_inputs = [text_preprocessing(txt) for txt in test_texts]

train_labels = corpus.train.labels
test_labels = corpus.test.labels

In [32]:
# 데이터셋을 피클로 저장.
import os
os.makedirs('data/nsmc')

train_data = {"text": train_inputs, "label": train_labels}
test_data = {"text": test_inputs, "label": test_labels}

import pickle
with open("data/nsmc/preprocessing_train.pkl", "wb") as fo:
    pickle.dump(train_data, fo)

with open("data/nsmc/preprocessing_test.pkl", "wb") as fo:
    pickle.dump(test_data, fo)

In [8]:
import pickle
with open("data/nsmc/preprocessing_train.pkl", "rb") as fi:
    train = pickle.load(fi)

with open("data/nsmc/preprocessing_test.pkl", "rb") as fi:
    test = pickle.load(fi)

train_inputs = train['text']
train_labels = train['label']

test_inputs = test['text']
test_labels = test['label']

In [9]:
all_inputs = train_inputs + test_inputs # list + list
# train/test set의 댓글들을 합치기. -> 토큰나이저(어휘사전) 생성을 위해.

### 토큰화
- Subword 방식 토큰화 적용
- Byte Pair Encoding 방식으로 huggingface tokenizer 사용
    - BPE: 토큰을 글자 단위로 나눈뒤 가장 자주 등장하는 글자 쌍(byte paire)를 찾아 합친뒤 어휘사전에 추가한다.
    - https://huggingface.co/docs/tokenizers/quicktour
    - `pip install tokenizers`

In [34]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

vocab_size = 30_000

tokenizer = Tokenizer(
    BPE(unk_token="<unk>")
)

tokenizer.pre_tokenizer = Whitespace()

trainer = BpeTrainer(
    vocab_size=vocab_size,
    min_frequency=5,
    special_tokens=["<pad>","<unk>"],
    continuing_subword_prefix="##" 
    # 시작 subword는 그대로. 연결 subword 앞에는 ##을 붙인다.
    # cowork : co, ##work
)

tokenizer.train_from_iterator(all_inputs, trainer=trainer)

# 학습데이터가 파일: tokenizer.train(["파일경로"])
# 학습 데이터가메모리에 iterable타입으로 있는 경우: tokenizer.train_from_iterator()

In [35]:
# 어휘사전 크기
tokenizer.get_vocab_size()

25639

In [ ]:
# 저장
tokenizer.save("saved_models/nsmc_bpe_tokenizer.json")
# 불러오기 : load_tokenizer = Tokenizer.from_file('경로')

In [5]:
from tokenizers import Tokenizer

tokenizer = Tokenizer.from_file("saved_models/nsmc_bpe_tokenizer.json")

In [42]:
# 인코딩 테스트
idx = 100
print(all_inputs[idx])

encode = tokenizer.encode(all_inputs[idx])

print(encode.tokens)
print(encode.ids)

신카이 마코토 의 작화 와 미유 와 하나카나 가 연기 를 잘 하다 어 주다 어서 더 대박 이다 였 다
['신카이', '마코토', '의', '작화', '와', '미', '##유', '와', '하나', '##카', '##나', '가', '연기', '를', '잘', '하다', '어', '주다', '어서', '더', '대박', '이다', '였', '다']
[20880, 19369, 2206, 8352, 2110, 1463, 3309, 2110, 5537, 3223, 3294, 528, 5466, 1319, 2234, 5444, 2033, 5457, 5450, 999, 5930, 5441, 2079, 962]


In [43]:
tokenizer.decode(encode.ids)

'신카이 마코토 의 작화 와 미 ##유 와 하나 ##카 ##나 가 연기 를 잘 하다 어 주다 어서 더 대박 이다 였 다'

In [46]:
print(encode.ids)

[20880, 19369, 2206, 8352, 2110, 1463, 3309, 2110, 5537, 3223, 3294, 528, 5466, 1319, 2234, 5444, 2033, 5457, 5450, 999, 5930, 5441, 2079, 962]


## Dataset, DataLoader 생성

In [52]:
train_inputs[5]

'막 걸음마 떼다 ᆫ 3 세 부터 초등학교 1 학년 생 이다 ᆫ 8 살 용 영화 ㅋㅋㅋ 별반 개 도 아깝다 음'

In [3]:
tokenizer.encode(train_inputs[5]).ids

NameError: name 'tokenizer' is not defined

In [11]:
tokenizer.token_to_id("<pad>")

0

In [15]:
max_length = 5
seq_len = 3
[1, 2, 3] + ([0] * (max_length-seq_len))

[1, 2, 3, 0, 0]

In [13]:
[0] * 3

[0, 0, 0]

In [21]:
# Dataset - Raw 데이터 셋에서 학습할 때 필요한 데이터를 하나씩 제공 역할.
#           subscriptable 타입의 클래스로 구현 (__len__(), __getitem__(indx) 둘을 구현)
#           Dataset객체[0]  0번 학습데이터를 제공. (x[0], y[0])
# DataLoader - Batch 단위로 묶어서 데이터를 제공.

import torch
from torch.utils.data import Dataset, DataLoader

class NSMCDataset(Dataset):
    
    def __init__(self, texts, labels, max_length, tokenizer):
        """
        texts: list - 댓글 리스트. 리스트에 댓글들을 담아서 받는다. ["댓글", "댓글", ...]
        labels: list - Label 리스트. (댓글의 긍부정 여부 - 긍정: 1, 부정: 0)
        max_length: 개별 댓글의 token 최대 개수. 모든 댓글의 토큰수를 max_length에 맞춘다.
        tokenizer: Tokenizer
        """
        self.max_length = max_length
        self.tokenizer = tokenizer
        self.labels = labels
        self.texts = [
            self.__pad_token_sequences(tokenizer.encode(txt).ids) for txt in texts
        ]    
        # 댓글 -> 토큰 ID, max_length 크기에 맞춤.(토큰수가 적으면 <pad>추가, 많으면 짤라내기.)

    ###########################################################################################
    # id로 구성된 개별 문장 token list를 받아서 패딩 추가 [20, 2, 1] => [20, 2, 1, 0, 0, 0, ..]
    ############################################################################################
    def __pad_token_sequences(self, token_sequences):
        """
        token id로 구성된 개별 문서(댓글)의 token_id list를 받아서 max_length 길이에 맞추는 메소드
        max_length 보다 토큰수가 적으면 <pad> 토큰 추가, 많으면 max_length 크기로 줄인다.
            ex) max_length=5 이고 pad토큰 id가 0이라면
                [20, 2, 1] => [20, 2, 1, 0, 0]
                [20, 21, 30, 34, 60, 17, 21, 33] -> [20, 21, 30, 34, 60]
        """
        # <pad>의 token id를 조회
        pad_token_id = self.tokenizer.token_to_id("<pad>")
        # 입력 받은 token_sequences의 토큰개수
        seq_len = len(token_sequences)
        # truncate 또는 padding 처리
        if self.max_length < seq_len: # truncate
            result = token_sequences[:self.max_length]
        else: # <pad> 추가
            result = token_sequences + ([pad_token_id] * (self.max_length - seq_len))

        return result
        
    def __len__(self):
        # 총 데이셋의 개수를 반환.
        return len(self.texts)


    def __getitem__(self, idx):
        """
        idx 번째 text와 label을 학습 가능한 type으로 변환해서 반환
        Parameter
            idx: int 조회할 index
        Return
            tuple: (torch.LongTensor, torch.FloatTensor) - 댓글 토큰_id 리스트, 정답 Label
        """
        comment = torch.tensor(self.texts[idx], dtype=torch.int64)   # 개별댓글 token id: dtype-int64(정수)
        label = torch.tensor([self.labels[idx]], dtype=torch.float32)   # label: dtype - float32
    
        return comment, label

In [22]:
max_length = 30
trainset = NSMCDataset(train_inputs, train_labels, max_length, tokenizer)
testset = NSMCDataset(test_inputs, test_labels, max_length, tokenizer)

len(trainset), len(testset)

(150000, 50000)

In [27]:
trainset[110]


(tensor([6495, 5450, 5555, 5460, 5442, 5443, 5739, 2055, 5477, 2048, 5487, 6285,
         2209, 6015, 5441,  847,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0]),
 tensor([0.]))

In [28]:
train_loader = DataLoader(trainset, batch_size=64, shuffle=True, drop_last=True)
test_loader = DataLoader(testset, batch_size=64)

In [29]:
len(train_loader), len(test_loader)

(2343, 782)

# 모델링
- Embedding Layer를 이용해 Word Embedding Vector를 추출한다.
- LSTM을 이용해 Feature 추출
- Linear + Sigmoid로 댓글 긍정일 확률 출력
  
![outline](figures/rnn/RNN_outline.png)

## 모델 정의

In [30]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [32]:
import torch
import torch.nn as nn

class NSMCClassifier(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers=1, bidirectional=True, dropout=0.2):
        super().__init__()
        # 모델 구성 Layer들: 
        ##   embedding layer(단어 임베딩)
        ##   lstm layer(문서/문장 임베딩)
        ##   linear(classifier) (분류)

        self.embedding = nn.Embedding(
            num_embeddings=vocab_size,   # num_embeddings x embedding_dim  
            embedding_dim=embedding_dim,
            padding_idx=0 # Padding 토큰의 idx지정. 이 index의 embedding vector는 학습하지 않는다.
        )
        # embedding_model의 출력 shape: (batch_size, seq_length, embedding_dim)
        
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            bidirectional=bidirectional,
            dropout=dropout
        )

        self.classifier = nn.Linear(
            in_features= hidden_size * 2 if bidirectional else hidden_size,
            out_features=1  # 이진분류의 출력 - 양성일 확률 1개
        )

        self.sigmoid = nn.Sigmoid()


    def forward(self, X):
        """
        Args: 
            X(torch.Tensor): 입력 문서의 토큰 리스트. shape: [batch_size, seq_length(max_length)], [64, 30]
            연산 순서 -> embedding_model =>(transpose)=>lstm => classifier => sigmoid
        """
        embedding_vector = self.embedding(X)
        #X: [batch_size, seq_length] -(embedding)-> ev: [batch_size, seq_length, embedding_dim]

        # batch_size 축과 seq_length 축의 index의 위치를 변경. index: [10, 5, 7] -> [5, 10, 7] 
        embedding_vector = embedding_vector.transpose(1, 0)

        # rnn/lstm/gru입력(batch_first=False): (seq_length, batch_size, embedding_dim)
        
        out, _ = self.lstm(embedding_vector)# out, (hidden, cell)
        # out.shape: [seq_length, batch, hidden_size*2 if bidirectiona else hidden_size]

        # 추론
        output = self.classifier(out[-1])
        last_output = self.sigmoid(output)
        return last_output


## 모델 생성

In [33]:
vocab_size = tokenizer.get_vocab_size()
embedding_dim = 100
hidden_size = 64
num_layers = 2
bidirectional = True
dropout = 0.3

In [34]:
!uv pip install torchinfo

Resolved 1 package in 241ms
Installed 1 package in 31ms
 + torchinfo==1.8.0


In [ ]:
# summary(모델, (100,784)) shape을 지정: input tensor type을 float 만들어서 실행.
# embedding 모델은 입력을 LongTensor(int64)를 입력을 받기 때문에 
# summary()에 직접 int타입 dummy data를 생성해서 전달


In [36]:
# torch info로 모델 확인
from torchinfo import summary
dummy_data = torch.randint(1, 10, (64, max_length))
summary(
    NSMCClassifier(vocab_size, embedding_dim, hidden_size, num_layers, bidirectional, dropout),
    input_data=dummy_data
)

Layer (type:depth-idx)                   Output Shape              Param #
NSMCClassifier                           [64, 1]                   --
├─Embedding: 1-1                         [64, 30, 100]             2,563,900
├─LSTM: 1-2                              [30, 64, 128]             184,320
├─Linear: 1-3                            [64, 1]                   129
├─Sigmoid: 1-4                           [64, 1]                   --
Total params: 2,748,349
Trainable params: 2,748,349
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 517.99
Input size (MB): 0.02
Forward/backward pass size (MB): 3.50
Params size (MB): 10.99
Estimated Total Size (MB): 14.51

## 학습

### Train/Test 함수 정의

In [37]:
# 1 에폭 학습 함수.
def train(model, dataloader, loss_fn, optimizer, device="cpu"):
    # 1. 모델을 train 모드로 변경
    model.train()
    # 2. 모델을 device로 이동
    model = model.to(device)

    # 1 에폭 학습
    train_loss = 0.0
    for X, y in dataloader:
        # 1 step 학습
        # 1. X, y를 device 이동
        X, y = X.to(device), y.to(device)

        # 2. 추론
        pred = model(X)

        # 3. loss 계산
        loss = loss_fn(pred, y)

        # 4. gradient 값 계산 - 오차역전파
        loss.backward()

        # 5. weight/bias(파라미터) 업데이트 = new_weight = weight.data - weight.grad * 학습율
        optimizer.step()

        # 6. grad 초기화
        optimizer.zero_grad()

        # loss값 누적
        train_loss += loss.item()

    return train_loss / len(dataloader) # 1 에폭 학습 loss를 반환.

In [48]:
@torch.no_grad
def eval(model, dataloader, loss_fn, device="cpu"):
    """모델 평가/검증 함수"""
    # 모델을 eval 모드로 변경. (평가/추론)
    model.eval()
    model.to(device)

    eval_loss, eval_acc = 0.0, 0.0
    
    for X,  y in dataloader:
        # 1. X, y를 device이동
        X, y = X.to(device), y.to(device)

        # 2. 추론  - 양성일 확률이 출력.
        pred_proba = model(X)
        pred_label = (pred_proba > 0.5).type(torch.int32)

        # 3. 평가(loss, accuracy)
        eval_loss += loss_fn(pred_proba, y).item()
        eval_acc += (pred_label == y).sum().item()  # 현재 step에서 몇개 맞았는지 대입.

    return eval_loss / len(dataloader), eval_acc / len(dataloader.dataset)
    # loss평균: step수로 나눔. accuracy평균: 총데이터개수.

### Train

In [49]:
lr = 0.0001
epochs = 3

model = NSMCClassifier(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    hidden_size=hidden_size,
    num_layers=num_layers,
    bidirectional=bidirectional,
    dropout=dropout
).to(device)

loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [50]:
from time import time
s = time()

train_loss_list = []
eval_loss_list = []
eval_acc_list = []

for epoch in range(epochs):
    train_loss = train(model, train_loader, loss_fn, optimizer, device)
    eval_loss, eval_acc = eval(model, test_loader, loss_fn, device)
    train_loss_list.append(train_loss)
    eval_loss_list.append(eval_loss)
    eval_acc_list.append(eval_acc)
    print(train_loss, eval_loss, eval_acc, sep=" || ")

e = time()
print("학습에 걸린 시간:", (e-s), "초")

0.5535337799286364 || 0.4423285055419673 || 0.79012
0.41571887614657826 || 0.4033110209400087 || 0.81414
0.38048250127362737 || 0.38546251027327977 || 0.8241
학습에 걸린 시간: 489.89029717445374 초


## 모델저장

In [51]:
torch.save(model, "saved_models/nsmc_lstm_model.pt")

In [52]:
load_model = torch.load("saved_models/nsmc_lstm_model.pt", weights_only=False)

# 서비스

## 전처리 함수들

In [53]:
# from konlpy.tag import Okt
from kiwipiepy import Kiwi
import string
import re

kiwi = Kiwi()
def text_preprocessing(text):
    """
    1. 영문 -> 소문자로 변환
    2. 구두점 제거
    3. 형태소 기반 토큰화
    4. 형태소로 토큰화 한 뒤 다시 하나의 문자열로 묶어서 반환.
    """
    text = text.lower()
    text = re.sub(rf"[{string.punctuation}]", ' ', text) # 구두점(특수문자)들을 ' '으로 변환.
    text = [token.lemma for token in kiwi.tokenize(text)] # [a, b, c,]
    return ' '.join(text)

In [54]:
def pad_token_sequences(token_sequences, max_length):
    """padding 처리 메소드."""
    pad_token = tokenizer.token_to_id('<pad>')  
    seq_length = len(token_sequences)           
    result = None
    if seq_length > max_length:                 
        result = token_sequences[:max_length]
    else:                                            
        result = token_sequences + ([pad_token] * (max_length - seq_length))
    return result

In [57]:
def predict_data_preprocessing(text_list):
    """
    모델에 입력할 수 있는 input data를 생성
    Parameter:
        text_list: list - 추론할 댓글리스트
    Return
        torch.LongTensor - 댓글 token_id tensor
    """
    # 기본 전처리
    text_list = [text_preprocessing(txt) for txt in text_list]
    # 토큰화
    token_list = [tokenizer.encode(txt).ids for txt in text_list]
    # 토큰개수 max_length에 맞추기
    token_list = [pad_token_sequences(token, max_length) for token in token_list]
    
    return torch.tensor(token_list, dtype=torch.int64)

## 추론

In [58]:
comment_list = ["아 진짜 재미없다.", "여기 식당 먹을만 해요", "이걸 영화라고 만들었냐?", "기대 안하고 봐서 그런지 괜찮은데.", "이걸 영화라고 만들었나?", "아! 뭐야 진짜.", "재미있는데.", "연기 짱 좋아. 한번 더 볼 의향도 있다.", "뭐 그럭저럭"]
input_tensor = predict_data_preprocessing(comment_list)
input_tensor.shape

torch.Size([9, 30])

In [60]:
input_tensor[1]

tensor([ 5896, 10957,  5693,  2194,  1345,  2890,  5458,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])

In [66]:
with torch.no_grad():
    pred_proba = model(input_tensor)
    pred_label = (pred_proba > 0.5).type(torch.int32)
    for txt, pred_label in zip(comment_list, pred_label):
        print(txt, end=" ||||||| ")
        print("긍정적 댓글" if pred_label.item()==1 else "부정적 댓글")

아 진짜 재미없다. ||||||| 부정적 댓글
여기 식당 먹을만 해요 ||||||| 부정적 댓글
이걸 영화라고 만들었냐? ||||||| 부정적 댓글
기대 안하고 봐서 그런지 괜찮은데. ||||||| 긍정적 댓글
이걸 영화라고 만들었나? ||||||| 부정적 댓글
아! 뭐야 진짜. ||||||| 긍정적 댓글
재미있는데. ||||||| 긍정적 댓글
연기 짱 좋아. 한번 더 볼 의향도 있다. ||||||| 긍정적 댓글
뭐 그럭저럭 ||||||| 부정적 댓글


In [64]:
pred_proba
pred_label

tensor([[0],
        [0],
        [0],
        [1],
        [0],
        [1],
        [1],
        [1],
        [0]], dtype=torch.int32)